In [15]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="c34a366b525d4ab59a6017a882071628",
    client_secret="cac3606c2df8478d8cd3c61a3e1698b6"))

In [ ]:
def get_songs_from_playlists(playlist_ids, num_songs):
    tracks = []
    for playlist_id in playlist_ids:
        playlist = sp.user_playlist_tracks("spotify", playlist_id)
        tracks.extend(playlist['items'])
        while sp.next(playlist):
            playlist = sp.next(playlist)
            tracks.extend(playlist['items'])
    
    songs = []
    for track in tracks:
        artist_id = track['track']['artists'][0]['id']
        artist_name = track['track']['artists'][0]['name']
        results = sp.artist_top_tracks(artist_id)
        for track in results['tracks'][:num_songs]:
            song_id = track['id']
            song_name = track['name']
            songs.append((song_id, song_name, artist_name))
    
    playlist_df = pd.DataFrame(songs, columns=['Song ID', 'Song Name', 'Artist Name'])
    return playlist_df

playlist_ids = ["4R8MaXWFRHH6bREubcphRg", "37i9dQZF1DX4WYpdgoIcn6", "7H3jsjFAOc1h4Ivs4PvfOH"]
num_songs = 30
playlist = get_songs_from_playlists(playlist_ids, num_songs)
playlist.to_csv('playlist_2.csv', index=False)


In [ ]:
playlist = pd.read_csv('playlist_2.csv')
playlist

In [ ]:
import time

audio_features_list = []

for song_id in playlist['Song ID']:
    audio_features = sp.audio_features(song_id)
    audio_features_list.append(audio_features)
    time.sleep(1)  

In [8]:
playlist_df = pd.json_normalize(audio_features_list)

[]

In [ ]:
data_frames = []

for item in audio_features_list:
    df = pd.DataFrame(item)  
    data_frames.append(df) 
    
final_df = pd.concat(data_frames, ignore_index=True)

In [ ]:
final_df.to_csv('playlist_audio.csv')

In [ ]:
playlist_num = final_df.select_dtypes(include='number')

In [ ]:
X_prep = StandardScaler().fit_transform(X)
pd.DataFrame(X_prep).head()

In [ ]:
K = range(2,20)
inertia =[]

for k in K:
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X_prep)
    inertia.append(kmeans.inertia_)

In [ ]:
kmeans= KMeans(n_clusters=5,random_state=1234)  
kmeans.fit(X_prep)
clusters = kmeans.predict(X_prep)
pd.Series(clusters).value_counts().sort_index()

In [ ]:
from sklearn.metrics import silhouette_score
K = range(2,20)
silhouette = []

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=1234)
    kmeans.fit(X_prep)
    silhouette.append(silhouette_score(X_prep,kmeans.predict(X_prep)))
    
    
plt.figure(figsize=(15,5))
plt.plot(K,silhouette)
plt.show()

In [ ]:
kmeans= KMeans(n_clusters=3,random_state=1234)  
kmeans.fit(X_prep)
clusters = kmeans.predict(X_prep)
pd.Series(clusters).value_counts().sort_index()

In [ ]:
import pickle
from sklearn.preprocessing import StandardScaler

# Replace the StandardScaler by the scaler of your choice
scaler = StandardScaler()
scaler.fit(X)


scaler_filename = 'scaler_sd.pkl'
pickle.dump(scaler, open(scaler_filename, 'wb'))


X_prep = scaler.transform(X)

model = KMeans(n_clusters=5,random_state=1234)
#This will take some time, especially if you have lots of columns
model.fit(X_prep)

model_filename = 'KMeans.plk'

pickle.dump(model, open(model_filename, 'wb'))

In [ ]:
kmeans= KMeans(n_clusters=5,random_state=1234)  
kmeans.fit(X_prep)
clusters = kmeans.predict(X_prep)
pd.Series(clusters).value_counts().sort_index()

In [ ]:
X_df= X
X_df['cluster'] = clusters
X_df

In [ ]:
X_df.to_csv('numerical.csv')

In [ ]:
playlist_clean = pd.read_csv('playlist_audio.csv',index_col=0)

In [ ]:
playlist_clean = playlist_clean.drop(playlist_clean.columns[-1], axis=1)

In [ ]:
playlist_clean = pd.concat([data, playlist_clean], axis=1)

In [ ]:
playlist_clean = playlist_clean.dropna()

In [ ]:
playlist_clean ['cluster'] = clusters

In [ ]:
 playlist_clean_2 = pd.concat([X_df, playlist_clean['Song Name']], axis=1)

In [ ]:
playlist_clean_2.to_csv('final_clean_df.csv')

In [ ]:
playlist_df = pd.read_csv('final_clean_df.csv',index_col=0)

In [ ]:
top100_df = pd.read_csv('Top_100_Billboard.csv',index_col=0)

In [ ]:
def check_top100(top100_df, model_file, scaler_file, playlist_df):
    song = input("Write a Song: ") # input song
    
    id_list = []   # list to store input song id
    audio_features_list = [] # list to store input song audio feature
    data_frames = [] # list to store the dataframes from the audio features list

    if song in top100_df['Title'].values:
        
        print("This song is present on top 100")  # foor loop to know if the input song is in the top 100
    else:
        client_id = 'ea97f40064524f9d812dfdc18ea50b86'
        client_secret = '3de672dadbed45b6a26b3d0db682fc1e'
        client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
        sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

        results = sp.search(q=song, limit=1, type='track')
        if len(results['tracks']['items']) > 0:
            song_id = results['tracks']['items'][0]['id']   # get the track ID
            id_list.append(song_id)   # add the track id to the list id_list
    for song_id in id_list:  # for loop to take the audio feature from the track id in the id_list
        audio_features = sp.audio_features(song_id)
        audio_features_list.append(audio_features)  # add the audio features to the list audio_features_list
        
    for item in audio_features_list: # for loop to take the audio feature from the audio_features list and transform to a dataframe
        df = pd.DataFrame(item)
        data_frames.append(df) # store the dataframes in the data_frames list
        
    final_df = pd.concat(data_frames, ignore_index=True) # concat all data frames in the data_frames list make only one data frame with all audio features
    playlist_num = final_df.select_dtypes(include='number') # take only the numeric data
    
    model_file = "KMeans.plk"
    scaler_file = "scaler_sd.pkl"
    with open(model_file, 'rb') as file:  #load the model from the pickle file
        model = pickle.load(file)
    with open(scaler_file, 'rb') as file: #load the scaler from the pickle file
        scaler = pickle.load(file)
        
        
 
        
    playlist_num_scaled = scaler.transform(playlist_num) # Apply the model and scaler to the numeric dataframe
    input_cluster = model.predict(playlist_num_scaled) # take the cluster from the input song
    
    
    matching_songs = []
    for index, cluster in enumerate(playlist_df['cluster']):  # for loop to compare the cluster from the input song with the cluster column from the playlist
        if cluster == input_cluster[0]:
            matching_songs.append(playlist_df.loc[index, 'Song Name']) # when the for loop find the matching cluster append the matching song to the list matching_songs
            
    recommended_song = random.choice(matching_songs) if matching_songs else None # find a a random song from the list and sugest a song for the same cluster to the user 
    
    
    return recommended_song

  
            

   

In [ ]:
recommended_song = check_top100(top100_df, "KMeans.pkl", "scaler.pkl", playlist_df)
print("The song is not present in the top 100, we recommend this song:", recommended_song)